In [ ]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join("../../")))
from src.recommendation_engine.recommend_best_device_per_group import recommend_best_per_group
from src.recommendation_engine.comparing_devices import comparing_devices
from src.comparisons.household import Kitchen, Bathroom, Room

from Frontend.src.components.plot_all_metrics_for_category_plotly import plot_all_metrics_for_category_plotly
import pandas as pd

import pickle
import matplotlib.pyplot as plt
import seaborn as sns

### Stworzenie słowników z urządzeniami odpowiednimi dla kazdego profilu

In [108]:
df = pd.read_parquet("../../Data/GRID/grid_with_profiles.parquet")
df[df.select_dtypes(include=['float']).columns] = df.select_dtypes(include=['float']).round(1)

# delete balanced profiles
df = df[df["profile"] != "Balanced"]
df.drop(columns=["optimal_score", "unique_parameter",], inplace=True)

unique_profiles = df["profile"].unique()
appliance_recommendations_for_profiles = {}

for profile in unique_profiles:
    appliance_recommendations_for_profiles[profile] = recommend_best_per_group(profile_label=profile, df=df, comparing_dict=comparing_devices)

with open("../../src/recommendation_engine/pickles/appliance_recommendations_for_profiles.pkl", "wb") as f:
    pickle.dump(appliance_recommendations_for_profiles, f)

In [115]:
with open("../../src/recommendation_engine/pickles/appliance_recommendations_for_profiles.pkl", "rb") as f:
    appliance_recommendations_for_profiles = pickle.load(f)

### Stworzenie df-a z porównaniem wszystkich urządzeń

In [89]:
df_water_boiling = Kitchen.compare_water_boiling_devices(liters=1.5)
df_cooking = Kitchen.compare_cooking_devices(time_minutes=30)
df_heating = Kitchen.compare_heating_devices(time_minutes=30)
df_coffee = Kitchen.compare_coffee_devices(cups=1)
df_robots = Kitchen.compare_multicookers(recipe_complexity=1.5)
df_heaters = Bathroom.compare_water_heaters(liters=50)
df_bath = Bathroom.compare_bathing_options()
df_air_heating = Bathroom.compare_bathroom_heating()
df_work = Room.compare_workstations()
df_cooling = Room.compare_cooling_devices(duration_min=60)

chosen_columns = ["name", "energy_kwh", "cost_pln", "time_min", "co2_emission_kg", "device_cost"]

In [90]:
# słownik z parametrami jakie wchodziły do funkcji, zeby wypisywac na tytule wykresu
category_assumptions = {
    'boiling_water': 'liters = 1.5',
    'cooking': 'time_minutes = 30',
    'heating_food': 'time_minutes = 30',
    'making_coffee': 'cups = 1',
    'multicookers': 'recipe_complexity = 1.5',
    'water_heating': 'liters = 50',
    'bathing': '',  # brak parametrów
    'bathroom_heating': '',  # brak parametrów
    'workstation': '',  # brak parametrów
    'cooling': 'duration_min = 60'
}

In [91]:
dfs_with_keys = [
    (df_water_boiling, 'boiling_water'),
    (df_cooking, 'cooking'),
    (df_heating, 'heating_food'),
    (df_coffee, 'making_coffee'),
    (df_robots, 'multicookers'),
    (df_heaters, 'water_heating'),
    (df_bath, 'bathing'),
    (df_air_heating, 'bathroom_heating'),
    (df_work, 'workstation'),
    (df_cooling, 'cooling')
]

combined_df = pd.concat([
    df[chosen_columns].assign(category=key) for df, key in dfs_with_keys
], ignore_index=True)

combined_df["time_min"] = round(combined_df["time_min"], 2)
combined_df["energy_kwh"] = round(combined_df["energy_kwh"], 2)
combined_df["co2_emission_kg"] = round(combined_df["co2_emission_kg"], 2)

In [114]:
with open("../../src/recommendation_engine/pickles/combined_df.pkl", "wb") as f:
    pickle.dump(combined_df, f)

In [ ]:
recommended = appliance_recommendations_for_profiles['Saver']                    # wybranie konkretnego profilu

category_order = list(recommended.keys())   # ustalenie kategorii do wykresów

# Filtruj tylko te kategorie
df_filtered = combined_df[combined_df['category'].isin(category_order)].copy()

# Oznacz urządzenia rekomendowane
df_filtered['is_recommended'] = df_filtered.apply(
    lambda row: row['name'] == recommended.get(row['category']), axis=1
)

In [113]:
import plotly.io as pio
for cat in category_order:
    fig = plot_all_metrics_for_category_plotly(df_filtered, cat, category_assumptions=category_assumptions)
    fig.show()  # lub zwróć jako output w callbacku Dash